# Fine-tuning Qwen2.5-0.5B para Epicrisis

Este notebook realiza fine-tuning del modelo Qwen2.5-0.5B-Instruct para generar epicrisis medicas.

**Requisitos:**
- GPU T4 o superior (disponible en Colab gratuito)
- ~8GB VRAM

**Dataset:**
- ~1200 ejemplos de epicrisis en formato ChatML
- 90% train / 10% validation

## 1. Configuracion del entorno

In [ ]:
# Verificar GPU disponible
!nvidia-smi

In [ ]:
# Instalar dependencias
!pip install -q transformers datasets accelerate peft bitsandbytes trl wandb
!pip install -q flash-attn --no-build-isolation

In [ ]:
import torch
import json
import os
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuracion

In [ ]:
# Configuracion del modelo y entrenamiento
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
OUTPUT_DIR = "./epicrisis-finetuned"
DATASET_DIR = "./datasets"

# Hiperparametros CONSERVADORES para evitar corromper el modelo
# El problema anterior fue que el modelo aprendio a generar prompts/JSON
# en lugar de solo respuestas narrativas

EPOCHS = 3  # 3 epochs es razonable con DataCollator correcto
BATCH_SIZE = 2
GRADIENT_ACCUMULATION = 4  # Effective batch size = 8
LEARNING_RATE = 2e-5  # Standard para LoRA fine-tuning
MAX_SEQ_LENGTH = 1024

# LoRA - configuracion conservadora
LORA_RANK = 16  # r=16 es bueno para tareas de generacion
LORA_ALPHA = 32  # alpha = 2*r es una buena regla
LORA_DROPOUT = 0.05

# System instruction (igual que en la app)
SYSTEM_INSTRUCTION = (
    "Genera una epicrisis narrativa en UN SOLO PARRAFO. "
    "USA SOLO la informacion del JSON, NO inventes datos. "
    "IMPORTANTE: Incluye TODOS los codigos entre parentesis: "
    "diagnostico de ingreso con codigo CIE-10 (ej: I20.0), "
    "procedimientos con codigo K (ej: K492, K493), "
    "medicacion con dosis y codigo ATC (ej: B01AC06). "
    "Estructura: dx ingreso -> procedimientos -> evolucion -> dx alta -> medicacion alta. "
    "Abreviaturas: DA=descendente anterior, CD=coronaria derecha, CX=circunfleja, "
    "SDST=supradesnivel ST, IAM=infarto agudo miocardio."
)

print("="*60)
print("CONFIGURACION DE FINE-TUNING")
print("="*60)
print(f"  Modelo: {MODEL_NAME}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"  Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Max seq length: {MAX_SEQ_LENGTH}")
print(f"  LoRA rank: {LORA_RANK}")
print(f"  LoRA alpha: {LORA_ALPHA}")
print(f"  LoRA dropout: {LORA_DROPOUT}")
print("="*60)
print("\nNOTA: Se usara DataCollatorForCompletionOnlyLM para")
print("      entrenar SOLO en las respuestas del assistant.")

## 3. Subir y preparar datasets

Sube los archivos JSONL del dataset o ejecuta la celda siguiente para crear datos de ejemplo.

In [ ]:
# Subir archivos del dataset unificado
# Sube los archivos train.jsonl y validation.jsonl de la carpeta unified_data/
from google.colab import files

os.makedirs(DATASET_DIR, exist_ok=True)

print("="*60)
print("IMPORTANTE: Sube los archivos del dataset unificado:")
print("  - unified_data/train.jsonl (1071 ejemplos)")
print("  - unified_data/validation.jsonl (120 ejemplos)")
print("="*60)
print()

uploaded = files.upload()

for filename, content in uploaded.items():
    # Guardar en el directorio de datasets
    filepath = f"{DATASET_DIR}/{filename}"
    with open(filepath, "wb") as f:
        f.write(content)
    
    # Contar lineas
    with open(filepath, "r", encoding="utf-8") as f:
        lines = sum(1 for _ in f)
    
    print(f"Guardado: {filepath} ({lines} ejemplos)")

In [ ]:
# Verificar archivos subidos
print("Archivos en el directorio de datasets:")
for f in Path(DATASET_DIR).glob("*.jsonl"):
    with open(f, "r", encoding="utf-8") as file:
        lines = sum(1 for _ in file)
    print(f"  - {f.name}: {lines} ejemplos")

In [ ]:
# Cargar datasets unificados
# Los archivos ya estan en formato ChatML (campo "text")

def load_unified_datasets(dataset_dir):
    """
    Carga los datasets unificados (train.jsonl y validation.jsonl).
    Estos archivos ya tienen el formato ChatML en el campo "text".
    """
    train_path = Path(dataset_dir) / "train.jsonl"
    valid_path = Path(dataset_dir) / "validation.jsonl"
    
    train_examples = []
    valid_examples = []
    
    # Cargar train
    if train_path.exists():
        with open(train_path, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    example = json.loads(line)
                    train_examples.append(example)
        print(f"Train: {len(train_examples)} ejemplos")
    else:
        raise FileNotFoundError(f"No se encontro {train_path}")
    
    # Cargar validation
    if valid_path.exists():
        with open(valid_path, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    example = json.loads(line)
                    valid_examples.append(example)
        print(f"Validation: {len(valid_examples)} ejemplos")
    else:
        raise FileNotFoundError(f"No se encontro {valid_path}")
    
    # Crear DatasetDict
    dataset = DatasetDict({
        "train": Dataset.from_list(train_examples),
        "validation": Dataset.from_list(valid_examples)
    })
    
    return dataset

# Cargar datasets
dataset = load_unified_datasets(DATASET_DIR)
print(f"\nDataset cargado: {dataset}")

# Verificar que tiene el campo "text"
print(f"\nCampos disponibles: {dataset['train'].column_names}")
if "text" in dataset['train'].column_names:
    print("✓ Campo 'text' encontrado (formato ChatML)")
else:
    print("✗ ERROR: No se encontro el campo 'text'")
    
# IMPORTANTE: Verificar longitud de los ejemplos
lengths = [len(ex["text"]) for ex in dataset["train"]]
print(f"\nEstadisticas de longitud (caracteres):")
print(f"  Min: {min(lengths)}")
print(f"  Max: {max(lengths)}")
print(f"  Promedio: {sum(lengths)/len(lengths):.0f}")

In [ ]:
# Ver ejemplos del dataset para verificar el formato
print("="*60)
print("VERIFICANDO FORMATO DEL DATASET")
print("="*60)

# Ver el primer ejemplo completo
example = dataset["train"][0]["text"]
print("\nEjemplo 1 (completo):")
print("-"*60)
print(example)
print("-"*60)

# Verificar que el formato es correcto
print("\n\nVerificaciones:")
print(f"1. Contiene '<|im_start|>system': {'<|im_start|>system' in example}")
print(f"2. Contiene '<|im_start|>user': {'<|im_start|>user' in example}")
print(f"3. Contiene '<|im_start|>assistant': {'<|im_start|>assistant' in example}")
print(f"4. Contiene '<|im_end|>': {'<|im_end|>' in example}")

# Contar tokens especiales
print(f"\n5. Numero de '<|im_start|>': {example.count('<|im_start|>')}")
print(f"6. Numero de '<|im_end|>': {example.count('<|im_end|>')}")

# Extraer y mostrar el output del assistant
if "<|im_start|>assistant\n" in example:
    assistant_part = example.split("<|im_start|>assistant\n")[1]
    if "<|im_end|>" in assistant_part:
        assistant_text = assistant_part.split("<|im_end|}")[0]
    else:
        assistant_text = assistant_part
    print(f"\n7. Output del assistant (primeros 500 chars):")
    print("-"*60)
    print(assistant_text[:500])

# Verificar todos los ejemplos tienen el formato correcto
print("\n" + "="*60)
print("VERIFICANDO TODOS LOS EJEMPLOS")
print("="*60)

errors = 0
for i, ex in enumerate(dataset["train"]):
    text = ex["text"]
    if "<|im_start|>assistant\n" not in text:
        print(f"  ERROR en ejemplo {i}: falta '<|im_start|>assistant\\n'")
        errors += 1
    if not text.strip().endswith("<|im_end|>"):
        print(f"  ERROR en ejemplo {i}: no termina con '<|im_end|>'")
        errors += 1

if errors == 0:
    print(f"✓ Todos los {len(dataset['train'])} ejemplos tienen formato correcto")
else:
    print(f"✗ Se encontraron {errors} errores")

In [ ]:
# Guardar datasets unificados para referencia
dataset["train"].to_json(f"{DATASET_DIR}/unified_train.jsonl")
dataset["validation"].to_json(f"{DATASET_DIR}/unified_validation.jsonl")
print("Datasets unificados guardados en:")
print(f"  - {DATASET_DIR}/unified_train.jsonl")
print(f"  - {DATASET_DIR}/unified_validation.jsonl")

## 4. Cargar modelo y tokenizer

In [ ]:
# Configuracion de cuantizacion (4-bit para ahorrar memoria)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# IMPORTANTE: Configurar pad_token correctamente para Qwen
# Qwen usa <|endoftext|> como eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Verificar tokens especiales de ChatML
print(f"Tokenizer cargado: {MODEL_NAME}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"EOS token: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")
print(f"PAD token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")

# Verificar que los tokens de ChatML existen
chatml_tokens = ["<|im_start|>", "<|im_end|>"]
print("\nTokens ChatML:")
for token in chatml_tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id == tokenizer.unk_token_id:
        print(f"  {token}: ⚠️ NO EXISTE (mapped to UNK)")
    else:
        print(f"  {token}: ID {token_id}")

In [ ]:
# Cargar modelo
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

# Preparar para entrenamiento con cuantizacion
model = prepare_model_for_kbit_training(model)

print(f"Modelo cargado: {MODEL_NAME}")
print(f"Parametros: {model.num_parameters():,}")

## 5. Configurar LoRA

In [ ]:
# Configuracion LoRA
lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

# Aplicar LoRA al modelo
model = get_peft_model(model, lora_config)

# Mostrar parametros entrenables
model.print_trainable_parameters()

## 6. Configurar entrenamiento

In [ ]:
# Configuracion de entrenamiento
# Los hiperparametros se configuran en SFTConfig en la siguiente celda

print("Configuracion de entrenamiento:")
print(f"  - Epochs: {EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"  - Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  - LoRA rank: {LORA_RANK}")
print(f"  - LoRA alpha: {LORA_ALPHA}")

In [ ]:
# Crear trainer para TRL >= 0.27.0
import trl
print(f"TRL version: {trl.__version__}")

from trl import SFTConfig, SFTTrainer

# Buscar DataCollatorForCompletionOnlyLM en diferentes ubicaciones de TRL
DataCollatorForCompletionOnlyLM = None

# Intentar diferentes ubicaciones
try:
    from trl import DataCollatorForCompletionOnlyLM
    print("Importado desde trl")
except ImportError:
    pass

if DataCollatorForCompletionOnlyLM is None:
    try:
        from trl.trainer import DataCollatorForCompletionOnlyLM
        print("Importado desde trl.trainer")
    except ImportError:
        pass

if DataCollatorForCompletionOnlyLM is None:
    try:
        from trl.trainer.utils import DataCollatorForCompletionOnlyLM
        print("Importado desde trl.trainer.utils")
    except ImportError:
        pass

if DataCollatorForCompletionOnlyLM is None:
    try:
        from trl.data_utils import DataCollatorForCompletionOnlyLM
        print("Importado desde trl.data_utils")
    except ImportError:
        pass

# Si no encontramos el collator, implementamos uno simple
if DataCollatorForCompletionOnlyLM is None:
    print("DataCollatorForCompletionOnlyLM no disponible en TRL 0.27.0")
    print("Usando implementacion alternativa con completion_only_collator...")
    
    from transformers import DataCollatorForLanguageModeling
    from dataclasses import dataclass
    from typing import Any, Dict, List
    import torch
    
    @dataclass
    class CompletionOnlyDataCollator:
        """
        Data collator que solo calcula loss en la parte del assistant.
        """
        tokenizer: Any
        response_template: str = "<|im_start|>assistant\n"
        mlm: bool = False
        
        def __call__(self, examples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
            # Tokenizar si es necesario
            if isinstance(examples[0], dict) and "text" in examples[0]:
                texts = [ex["text"] for ex in examples]
                batch = self.tokenizer(
                    texts,
                    padding=True,
                    truncation=True,
                    max_length=1024,
                    return_tensors="pt",
                )
            else:
                batch = self.tokenizer.pad(examples, return_tensors="pt")
            
            # Crear labels (copiar input_ids)
            labels = batch["input_ids"].clone()
            
            # Tokenizar el response_template
            response_token_ids = self.tokenizer.encode(
                self.response_template, 
                add_special_tokens=False
            )
            
            # Para cada ejemplo, enmascarar todo antes del response_template
            for i in range(labels.shape[0]):
                input_ids = batch["input_ids"][i].tolist()
                
                # Buscar donde empieza la respuesta del assistant
                response_start = None
                for j in range(len(input_ids) - len(response_token_ids) + 1):
                    if input_ids[j:j+len(response_token_ids)] == response_token_ids:
                        response_start = j + len(response_token_ids)
                        break
                
                # Si encontramos el template, enmascarar todo antes
                if response_start is not None:
                    labels[i, :response_start] = -100
                
                # Tambien enmascarar padding
                labels[i, batch["attention_mask"][i] == 0] = -100
            
            batch["labels"] = labels
            return batch
    
    DataCollatorForCompletionOnlyLM = CompletionOnlyDataCollator

# Crear el collator
response_template = "<|im_start|>assistant\n"
collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template=response_template,
)

print(f"\nResponse template: '{response_template}'")
print(f"Response template tokens: {tokenizer.encode(response_template, add_special_tokens=False)}")

sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    eval_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    gradient_checkpointing=True,
    max_grad_norm=0.5,
    # Parametros especificos de SFT (TRL 0.27+)
    max_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    data_collator=collator,
)

print("\nTrainer configurado")
print("  - Solo entrena en las respuestas del assistant")
print("  - Evita que el modelo 'aprenda' a generar prompts")

## 7. Entrenar modelo

In [ ]:
# Entrenar
print("Iniciando entrenamiento...")
print("="*60)

trainer.train()

print("="*60)
print("Entrenamiento completado!")

In [ ]:
# Guardar modelo final
trainer.save_model(f"{OUTPUT_DIR}/final")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final")

print(f"Modelo guardado en: {OUTPUT_DIR}/final")

## 8. Probar modelo

In [ ]:
# Probar generacion
from transformers import pipeline

def generate_epicrisis(model, tokenizer, input_data, max_new_tokens=512):
    """
    Genera una epicrisis dado un input JSON.
    """
    json_str = json.dumps(input_data, ensure_ascii=False, indent=2)
    
    prompt = (
        f"<|im_start|>system\n{SYSTEM_INSTRUCTION}<|im_end|>\n"
        f"<|im_start|>user\n{json_str}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    print(f"Prompt length: {len(prompt)} chars")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    print(f"Input tokens: {inputs['input_ids'].shape[1]}")
    
    # Tokens especiales de Qwen para terminar generacion
    eos_token_id = tokenizer.eos_token_id
    im_end_id = tokenizer.convert_tokens_to_ids("<|im_end|>")
    endoftext_id = tokenizer.convert_tokens_to_ids("<|endoftext|>")
    
    stop_ids = [eos_token_id]
    if im_end_id and im_end_id != tokenizer.unk_token_id:
        stop_ids.append(im_end_id)
    if endoftext_id and endoftext_id != tokenizer.unk_token_id:
        stop_ids.append(endoftext_id)
    
    print(f"EOS token ID: {eos_token_id}")
    print(f"im_end token ID: {im_end_id}")
    print(f"Stop IDs: {stop_ids}")
    
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=max_new_tokens,
            min_new_tokens=100,  # Forzar minimo de tokens
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=stop_ids,
            repetition_penalty=1.15,
            no_repeat_ngram_size=3,
        )
    
    # Decodificar solo los tokens nuevos
    input_length = inputs['input_ids'].shape[1]
    generated_tokens = outputs[0][input_length:]
    print(f"Generated {len(generated_tokens)} new tokens")
    
    # Decodificar
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    print(f"Decoded response length: {len(response)} chars")
    
    return response.strip()


def generate_with_pipeline(model, tokenizer, input_data):
    """Genera usando pipeline de transformers."""
    json_str = json.dumps(input_data, ensure_ascii=False, indent=2)
    
    messages = [
        {"role": "system", "content": SYSTEM_INSTRUCTION},
        {"role": "user", "content": json_str}
    ]
    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        return_full_text=False,
    )
    
    result = pipe(messages)
    return result[0]["generated_text"]

print("Funciones de generacion definidas")

In [ ]:
# Ejemplo de prueba
test_input = {
    "dx": ["Angina inestable (I20.0)"],
    "proc": ["Coronariografia (K492)", "Angioplastia DA (K493)"],
    "tto": [
        "Aspirina 300mg carga (B01AC06)",
        "Enoxaparina 60mg SC c/12h (B01AB05)",
    ],
    "evo": "SDST V1-V4. Oclusion DA proximal. Angioplastia exitosa con stent.",
    "dx_alta": ["IAM pared anterior (I21.0)"],
    "med": [
        "Aspirina 100mg VO c/24h (B01AC06)",
        "Clopidogrel 75mg VO c/24h 12m (B01AC04)",
    ],
}

print("Input:")
print(json.dumps(test_input, indent=2, ensure_ascii=False))
print("\n" + "="*60 + "\n")

# Probar con el modelo base (sin fine-tuning) para comparar
print("Probando con modelo BASE (Qwen2.5-0.5B-Instruct sin fine-tuning)...")
print("Cargando modelo base...")

from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_test = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)
base_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

json_str = json.dumps(test_input, ensure_ascii=False, indent=2)
messages = [
    {"role": "system", "content": SYSTEM_INSTRUCTION},
    {"role": "user", "content": json_str}
]

text = base_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = base_tokenizer(text, return_tensors="pt").to(base_model_test.device)

with torch.no_grad():
    outputs = base_model_test.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

response = base_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print("\nRespuesta del modelo BASE:")
print("="*60)
print(response)

# Liberar memoria
del base_model_test
torch.cuda.empty_cache()

print("\n" + "="*60)
print("\nAhora probando con modelo FINE-TUNED...")
print("="*60)

response_ft = generate_epicrisis(model, tokenizer, test_input)
print("\nRespuesta del modelo FINE-TUNED:")
print("="*60)
print(response_ft)

## 9. Fusionar y exportar modelo completo

In [ ]:
# Fusionar LoRA con modelo base
from peft import PeftModel

# Cargar modelo base sin cuantizacion para fusion
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

# Cargar adaptadores LoRA
merged_model = PeftModel.from_pretrained(base_model, f"{OUTPUT_DIR}/final")

# Fusionar
merged_model = merged_model.merge_and_unload()

print("Modelo fusionado")

In [ ]:
# Guardar modelo fusionado
MERGED_OUTPUT = f"{OUTPUT_DIR}/merged"

merged_model.save_pretrained(MERGED_OUTPUT)
tokenizer.save_pretrained(MERGED_OUTPUT)

print(f"Modelo fusionado guardado en: {MERGED_OUTPUT}")

## 10. Exportar a ONNX (opcional)

In [ ]:
# Instalar optimum para exportar a ONNX
!pip install -q optimum[exporters] onnx onnxruntime

In [ ]:
# Exportar a ONNX
from optimum.onnxruntime import ORTModelForCausalLM

ONNX_OUTPUT = f"{OUTPUT_DIR}/onnx"

# Exportar
ort_model = ORTModelForCausalLM.from_pretrained(
    MERGED_OUTPUT,
    export=True,
    provider="CPUExecutionProvider",
)

ort_model.save_pretrained(ONNX_OUTPUT)
tokenizer.save_pretrained(ONNX_OUTPUT)

print(f"Modelo ONNX guardado en: {ONNX_OUTPUT}")

## 11. Descargar modelo

In [ ]:
# Comprimir y descargar modelo fusionado
!zip -r epicrisis-merged.zip {MERGED_OUTPUT}

from google.colab import files
files.download("epicrisis-merged.zip")

In [ ]:
# Comprimir y descargar modelo ONNX (si se exporto)
if os.path.exists(ONNX_OUTPUT):
    !zip -r epicrisis-onnx.zip {ONNX_OUTPUT}
    files.download("epicrisis-onnx.zip")

## Resumen

Este notebook realiza:

1. **Carga y unificacion de datasets** - Combina todos los archivos JSONL en un solo dataset con formato ChatML
2. **Fine-tuning con LoRA** - Entrena el modelo Qwen2.5-0.5B-Instruct con cuantizacion 4-bit
3. **Fusion del modelo** - Combina los adaptadores LoRA con el modelo base
4. **Exportacion a ONNX** - Para uso en el navegador

### Archivos generados:
- `datasets/unified_train.jsonl` - Dataset de entrenamiento unificado
- `datasets/unified_validation.jsonl` - Dataset de validacion unificado
- `epicrisis-finetuned/final/` - Adaptadores LoRA
- `epicrisis-finetuned/merged/` - Modelo fusionado completo
- `epicrisis-finetuned/onnx/` - Modelo en formato ONNX